In [1]:
!pip install fastai==2.7.9
!pip install fastkaggle==0.0.7
!pip install ipython-autotime==0.3.1
!pip install torch==1.12.1
!pip install fairscale==0.4.6
%load_ext autotime

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.5/225.5 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.5/887.5 MB 768.1 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 51.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 57.5 MB/s eta 0:00:00
  Attempting uninstall: torch
    Found existing installation: torch 2.0.0
    Uninstalling torch-2.0.0:
      Successfully uninstalled torch-2.0.0
  Attempting uninstall: fastai
    Found existing installation: fastai 2.7.12
    Uninstalling fastai-2.7.12:
      Successfully uninstalled fastai-2.7.12
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchdata 0.6.0 requires torch==2.0.

In [2]:
import numpy as np
from fairscale.experimental.tooling.layer_memory_tracker import LayerwiseMemoryTracker
from fastai.vision.all import *

/opt/conda/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/opt/conda/lib/python3.10/site-packages/torchvision/image.so: undefined symbol: _ZN3c104cuda20CUDACachingAllocator9allocatorE'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


time: 3.6 s (started: 2023-05-29 11:54:30 +00:00)


In [3]:
# Set the size of the image
img_size = (64, 64)

def create_random_data():
    # Generate a random NumPy array
    rand_arr = np.random.rand(*img_size, 3) * 255
    rand_arr = rand_arr.astype(np.uint8)
    return rand_arr

def get_model_file_size(model): 
    param_size = 0
    for param in model.parameters():
        param_size += param.nelement() * param.element_size()

    model_size = param_size
    print('Model size: {} Byte'.format(model_size))

# Create your learner and load your model
model = load_learner("/kaggle/input/testingpklforinference/VGG16_bn_fp16_accum2.pkl")

get_model_file_size(model.model)

# Set the model to evaluation mode
model.eval()

# Get activation memory
tracker = LayerwiseMemoryTracker()
tracker.monitor(model)

x = create_random_data()
preds = model.predict(x)


tracker.stop()


Model size: 537239848 Byte


time: 5.94 s (started: 2023-05-29 11:54:34 +00:00)


In [4]:
forward_act = sum(t.event.memory_activations for t in tracker.forward_traces) 
print(f"Activation Memory: {forward_act} Byte")

Activation Memory: 9480232 Byte
time: 737 µs (started: 2023-05-29 11:54:40 +00:00)
